<a href="https://colab.research.google.com/github/Dicere/WB_Internship/blob/main/0_5_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext

In [ ]:
!pip install catboost
!pip install lazypredict

In [ ]:
!pip install nltk
!pip install pymorphy2
!pip string
!pip install emoji
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install pyspellchecker
!pip install deeppavlov
!pip install transformers

In [9]:
import nltk
import string
import pymorphy2
nltk.download('stopwords')
nltk.download('punkt')
nltk.download("wordnet")
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import collections
import re
from spellchecker import SpellChecker
from nltk.sentiment import SentimentIntensityAnalyzer
from deeppavlov import build_model, configs
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizerFast
import torch
from transformers import AutoModelForSequenceClassification
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from catboost import CatBoostClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report, recall_score, precision_recall_curve,precision_score, precision_recall_fscore_support
import statsmodels.api as sm
from sklearn.model_selection import StratifiedKFold
from lazypredict.Supervised import LazyClassifier

In [11]:
df = pd.read_csv('/content/drive/MyDrive/WB_стажировка/wb_school_task_2.csv.gzip',compression='gzip')
df = df.drop_duplicates(ignore_index=True)

In [12]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)
morph = pymorphy2.MorphAnalyzer()
tqdm.pandas()

In [13]:
stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()
em= ['☝️', '☹', '☺️', '✌', '✨', '❤', '❤️', '❤️\u200d🔥', '🌸', '🌹', '🌺', '🌼', '🎁', '🎄', '🐈\u200d⬛', '🐤', '🐰', '👌', '👌🏻', '👌🏼', '👍', '👍🏻', '👍🏼', '👍🏽', '👍🏿', '👎🏼', '👏', '👾', '💄', '💋', '💐', '💓', '💕', '💖', '💗', '💘', '💙', '💛', '💜', '💞', '💣', '💥', '💩', '💪', '💫', '💯', '🔥', '😀', '😁', '😂', '😃', '😅', '😉', '😊', '😋', '😌', '😍', '😏', '😒', '😔', '😘', '😜', '😞', '😡', '😢', '😣', '😭', '😼', '😿', '🙂', '🙈', '🙌', '🙌🏻', '🙍', '🙏', '🛁', '🤍', '🤔', '🤗', '🤙', '🤣', '🤤', '🤦\u200d♀️', '🤩', '🤪', '🤬', '🤮', '🤷\u200d♀️', '🥰', '🥲', '🥳', '🥶', '🥺', '🪔', '🫒', '\U0001fae7', '\U0001faf6🏻']
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if char not in em])
    tokens = text.split()
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]
    return ' '.join(tokens)

df['text_preproc'] = df['text'].apply(preprocess_text)
text = ' '.join(df[df['label']==0].text_preproc)

In [14]:
df['label_for_fast_text'] = '__label__' + df['label'].astype(str)

In [18]:
cat_feat = ['text']
feature=df[df.columns[3:]].columns

In [20]:
X = df[feature].drop(columns=['label','text',])
y = df['label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [29]:
X_train[['label_for_fast_text', 'text_preproc']].to_csv('fasttext_train.txt', index=False, sep=' ', header=False)
X_test[['label_for_fast_text', 'text_preproc']].to_csv('fasttext_test.txt', index=False, sep=' ', header=False)

In [30]:
import fasttext

In [31]:
model = fasttext.train_supervised('/content/fasttext_train.txt')

In [35]:
result = model.test('/content/fasttext_test.txt')

In [37]:
print(f"Precision: {result[1]}")
print(f"Recall: {result[2]}")

Precision: 0.736
Recall: 0.736


In [47]:
result = model.test('/content/fasttext_train.txt')

In [44]:
prediction = model.predict('довольный покупка спасибо',k=2)

print(f"Predicted label: {prediction[0][0]}")
print(f"Probability: {prediction[1][0]}")

Predicted label: __label__0
Probability: 0.9328545331954956


In [57]:
def text_score(x):
  pred = model.predict(x,k=2)
  return pred[1][1]

In [58]:
df['label_1'] = df['text_preproc'].apply(lambda x:text_score(x))

In [76]:
cat_feat = ['text']
feature=df[df.columns[3:]].columns

In [104]:
X = df[feature].drop(columns=['text','label','label_for_fast_text','text_preproc',])
# X = df['label_1']
y = df['label']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [106]:
def predict_precsion_recall(y_test, pred):
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  return [precision, recall]

In [107]:
clf = LazyClassifier(verbose=0,ignore_warnings=False, custom_metric=predict_precsion_recall)
models, predictions = clf.fit(X_train[X_train.columns[1:]], X_test[X_test.columns[1:]], y_train, y_test)
models[['Precision', 'Recall']] = models['predict_precsion_recall'].apply(lambda x: pd.Series(x))
models.drop(columns=['predict_precsion_recall'],inplace=True)
models.sort_values(by='F1 Score', ascending=False,inplace=True)
models

 21%|██        | 6/29 [00:03<00:11,  2.07it/s]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)


 90%|████████▉ | 26/29 [00:10<00:00,  3.75it/s]

StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'


100%|██████████| 29/29 [00:12<00:00,  2.25it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision,Recall
Model,,,,,,,
LGBMClassifier,0.74,0.60,0.60,0.71,0.50,0.51,0.30
ExtraTreesClassifier,0.75,0.59,0.59,0.71,1.28,0.54,0.26
XGBClassifier,0.73,0.59,0.59,0.71,2.03,0.49,0.30
RandomForestClassifier,0.73,0.58,0.58,0.70,1.00,0.49,0.25
BernoulliNB,0.74,0.57,0.57,0.70,0.06,0.51,0.23
AdaBoostClassifier,0.73,0.58,0.58,0.70,1.20,0.48,0.26
KNeighborsClassifier,0.72,0.57,0.57,0.69,0.13,0.45,0.24
ExtraTreeClassifier,0.70,0.59,0.59,0.69,0.08,0.41,0.36
SGDClassifier,0.72,0.57,0.57,0.69,0.07,0.43,0.25


In [112]:
clf = CatBoostClassifier(verbose=0)
clf.fit(X_train, y_train)

In [113]:
threshold = 0.5

In [114]:
y_pred = clf.predict_proba(X_test)[:, 1]
y_pred_metki = (y_pred> threshold).astype('int')

In [115]:
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_metki, average='macro')
roc = roc_auc_score(y_test, y_pred)
print("AUC_ROC----"+str(roc))
print("F1-score-----"+str(f1))
print("Precision-----"+str(precision))
print("Recall-----"+str(recall))

AUC_ROC----0.7284189723320159
F1-score-----0.6420487086481033
Precision-----0.6952358055856072
Recall-----0.6283926218708827
